# Sanofi
## Beachten
* Somehow, table recognision doesnt work. Used template function in tabula
* Type by text Recognition
* Names with "von" going wrong during reverting

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [77]:
# Read pdf into DataFrame
a = [69.99988674407959, 40.00007697631834, 559.4123867440796, 800.9575769763184]
df = tabula.read_pdf("Sanofi-Aventis-Schweiz-Transparenzliste-2017.pdf", pages='all', lattice=True, multitable=True, area=a)

## Format Table

In [78]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']


#Shift all where 'name' is empty
df_export[df_export['empty0'].notna()] = df_export[df_export['empty0'].isna()].shift(-1, axis='columns')

#Add Type
index_hco = df_export[df_export['name'].str.contains("INDIVIDUAL NAMED DISCLOSURE", na=False)].index[1]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove empty rows
df_export = df_export.dropna(subset=['total'], how='all')

#Remove empties
df_export.drop(columns=['empty0', 'empty1'], inplace=True)

#Remove Carinations
df_export = remove_carination(df_export, " ")

#Revert name
df_export.loc[df_export.type == 'hcp', 'name'] = revert_name(df_export[df_export.type == 'hcp']['name'], ' ')

#Fix Revert name
df_export['name'] = df_export['name'].str.replace('Weid Der Von', 'Von Der Weid')
df_export['name'] = df_export['name'].str.replace('Burg Von', 'Von Burg')

#Cleanup Numbers and sum
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Add empty fields
df_export = add_plz(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'sanofi')

revert_name: Be sure: Only revert hcp, not hco!
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')